# רשתות עצביות חוזרות

במודול הקודם, עסקנו בייצוגים סמנטיים עשירים של טקסט. הארכיטקטורה שבה השתמשנו תופסת את המשמעות המצטברת של מילים במשפט, אך היא אינה מתחשבת **בסדר** המילים, מכיוון שפעולת האגרגציה שמתרחשת לאחר ההטבעות מסירה את המידע הזה מהטקסט המקורי. מכיוון שמודלים אלו אינם מסוגלים לייצג את סדר המילים, הם אינם יכולים לפתור משימות מורכבות או עמומות יותר כמו יצירת טקסט או מענה על שאלות.

כדי לתפוס את המשמעות של רצף טקסט, נשתמש בארכיטקטורה של רשת עצבית הנקראת **רשת עצבית חוזרת**, או RNN. כאשר משתמשים ב-RNN, אנו מעבירים את המשפט דרך הרשת, טוקן אחד בכל פעם, והרשת מייצרת **מצב** מסוים, אותו אנו מעבירים שוב לרשת יחד עם הטוקן הבא.

![תמונה המציגה דוגמה ליצירת רשת עצבית חוזרת.](../../../../../translated_images/he/rnn.27f5c29c53d727b5.webp)

בהינתן רצף הטוקנים $X_0,\dots,X_n$, ה-RNN יוצר רצף של בלוקים של רשת עצבית, ומאמן את הרצף הזה מקצה לקצה באמצעות שיטת ה-backpropagation. כל בלוק ברשת מקבל זוג $(X_i,S_i)$ כקלט, ומייצר $S_{i+1}$ כתוצאה. המצב הסופי $S_n$ או הפלט $Y_n$ מועבר למסווג ליניארי כדי לייצר את התוצאה. כל בלוקי הרשת חולקים את אותם משקלים, ומאומנים מקצה לקצה באמצעות מעבר אחד של backpropagation.

> התרשים לעיל מציג רשת עצבית חוזרת בצורה "פרוסה" (משמאל), ובייצוג חוזר קומפקטי יותר (מימין). חשוב להבין שכל תאי ה-RNN חולקים את אותם **משקלים ניתנים לשיתוף**.

מכיוון שוקטורי המצב $S_0,\dots,S_n$ מועברים דרך הרשת, ה-RNN מסוגל ללמוד תלות רציפה בין מילים. לדוגמה, כאשר המילה *לא* מופיעה במקום כלשהו ברצף, הרשת יכולה ללמוד כיצד לשלול אלמנטים מסוימים בתוך וקטור המצב.

בתוך כל תא RNN ישנם שני מטריצות משקל: $W_H$ ו-$W_I$, והטיה $b$. בכל שלב של RNN, בהינתן קלט $X_i$ ומצב קלט $S_i$, מצב הפלט מחושב כ-$S_{i+1} = f(W_H\times S_i + W_I\times X_i+b)$, כאשר $f$ היא פונקציית הפעלה (לעיתים $\tanh$).

> עבור בעיות כמו יצירת טקסט (שנעסוק בהן ביחידה הבאה) או תרגום מכונה, אנו גם רוצים לקבל ערך פלט בכל שלב של RNN. במקרה כזה, ישנה גם מטריצה נוספת $W_O$, והפלט מחושב כ-$Y_i=f(W_O\times S_i+b_O)$.

בואו נראה כיצד רשתות עצביות חוזרות יכולות לעזור לנו לסווג את מערך הנתונים של החדשות שלנו.

> עבור סביבת הסנדבוקס, יש להריץ את התא הבא כדי לוודא שהספרייה הנדרשת מותקנת, והנתונים נטענים מראש. אם אתם עובדים באופן מקומי, ניתן לדלג על התא הבא.


In [1]:
import sys
!{sys.executable} -m pip install --quiet tensorflow_datasets==4.4.0
!cd ~ && wget -q -O - https://mslearntensorflowlp.blob.core.windows.net/data/tfds-ag-news.tgz | tar xz

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

# We are going to be training pretty large models. In order not to face errors, we need
# to set tensorflow option to grow GPU memory allocation when required
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

ds_train, ds_test = tfds.load('ag_news_subset').values()

כאשר מאמנים מודלים גדולים, הקצאת זיכרון GPU עשויה להפוך לבעיה. ייתכן שנצטרך גם להתנסות עם גדלים שונים של מיניבאצ'ים, כך שהנתונים יתאימו לזיכרון ה-GPU שלנו, ובכל זאת האימון יהיה מהיר מספיק. אם אתם מריצים את הקוד הזה על מכונת GPU משלכם, תוכלו להתנסות בהתאמת גודל המיניבאצ' כדי להאיץ את האימון.

> **Note**: ידוע כי גרסאות מסוימות של מנהלי התקן NVidia אינן משחררות את הזיכרון לאחר אימון המודל. אנחנו מריצים מספר דוגמאות במחברת זו, וזה עלול לגרום למיצוי הזיכרון בתצורות מסוימות, במיוחד אם אתם מבצעים ניסויים משלכם כחלק מאותה מחברת. אם אתם נתקלים בשגיאות מוזרות כאשר אתם מתחילים לאמן את המודל, ייתכן שתרצו להפעיל מחדש את ליבת המחברת.


In [3]:
batch_size = 16
embed_size = 64

## מסווג RNN פשוט

במקרה של RNN פשוט, כל יחידה חוזרת היא רשת לינארית פשוטה, אשר מקבלת וקטור קלט ווקטור מצב, ומפיקה וקטור מצב חדש. ב-Keras, ניתן לייצג זאת באמצעות השכבה `SimpleRNN`.

למרות שניתן להעביר טוקנים מקודדים ב-one-hot ישירות לשכבת ה-RNN, זו אינה גישה מומלצת בשל הממדיות הגבוהה שלהם. לכן, נשתמש בשכבת embedding להקטנת הממדיות של וקטורי המילים, ולאחר מכן בשכבת RNN, ולבסוף במסווג `Dense`.

> **הערה**: במקרים שבהם הממדיות אינה כה גבוהה, לדוגמה כאשר משתמשים בטוקניזציה ברמת תווים, ייתכן שיהיה הגיוני להעביר טוקנים מקודדים ב-one-hot ישירות לתא ה-RNN.


In [4]:
vocab_size = 20000

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=vocab_size,
    input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 64)          1280000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 16)                1296      
_________________________________________________________________
dense (Dense)                (None, 4)                 68        
Total params: 1,281,364
Trainable params: 1,281,364
Non-trainable params: 0
_________________________________________________________________


> **הערה:** כאן אנו משתמשים בשכבת הטמעה לא מאומנת לצורך פשטות, אך כדי לקבל תוצאות טובות יותר ניתן להשתמש בשכבת הטמעה מאומנת מראש באמצעות Word2Vec, כפי שתואר ביחידה הקודמת. זה יכול להיות תרגיל טוב עבורך להתאים את הקוד כך שיעבוד עם הטמעות מאומנות מראש.

עכשיו בואו נתחיל לאמן את ה-RNN שלנו. באופן כללי, קשה מאוד לאמן RNNs, מכיוון שברגע שתאי ה-RNN נפרסים לאורך רצף, מספר השכבות המעורבות בתהליך ה-backpropagation הופך להיות גדול מאוד. לכן, עלינו לבחור קצב למידה קטן יותר ולאמן את הרשת על מערך נתונים גדול יותר כדי לקבל תוצאות טובות. זה יכול לקחת זמן רב, ולכן מומלץ להשתמש ב-GPU.

כדי לזרז את התהליך, נאמן את מודל ה-RNN רק על כותרות חדשות, ונשמיט את התיאור. אתה יכול לנסות לאמן עם התיאור ולראות אם תצליח לגרום למודל להתאמן.


In [5]:
def extract_title(x):
    return x['title']

def tupelize_title(x):
    return (extract_title(x),x['label'])

print('Training vectorizer')
vectorizer.adapt(ds_train.take(2000).map(extract_title))

Training vectorizer


In [6]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize_title).batch(batch_size),validation_data=ds_test.map(tupelize_title).batch(batch_size))

7500/7500 [==============================] - 82s 11ms/step - loss: 0.6629 - acc: 0.7623 - val_loss: 0.5559 - val_acc: 0.7995


> **שימו לב** שהדיוק עשוי להיות נמוך יותר כאן, מכיוון שאנחנו מתאמנים רק על כותרות חדשות.


## חזרה על רצפי משתנים

זכרו שהשכבה `TextVectorization` תוסיף באופן אוטומטי סימני ריפוד לרצפים באורך משתנה בתוך קבוצת מיניבאץ'. מסתבר שסימנים אלו משתתפים גם בתהליך האימון, והם יכולים להקשות על התכנסות המודל.

ישנם מספר גישות שניתן לנקוט כדי לצמצם את כמות הריפוד. אחת מהן היא לסדר מחדש את מערך הנתונים לפי אורך הרצף ולחלק את כל הרצפים לפי גודל. ניתן לעשות זאת באמצעות הפונקציה `tf.data.experimental.bucket_by_sequence_length` (ראו [תיעוד](https://www.tensorflow.org/api_docs/python/tf/data/experimental/bucket_by_sequence_length)).

גישה נוספת היא להשתמש ב**מסכות**. ב-Keras, יש שכבות שתומכות בקלט נוסף שמראה אילו סימנים יש לקחת בחשבון במהלך האימון. כדי לשלב מסכות במודל שלנו, ניתן להוסיף שכבת `Masking` נפרדת ([תיעוד](https://keras.io/api/layers/core_layers/masking/)), או להגדיר את הפרמטר `mask_zero=True` בשכבת ה-`Embedding` שלנו.

> **Note**: האימון הזה ייקח בערך 5 דקות להשלמת אפוק אחד על כל מערך הנתונים. אתם מוזמנים להפסיק את האימון בכל רגע אם תתייאשו. מה שניתן לעשות גם הוא להגביל את כמות הנתונים המשמשים לאימון, על ידי הוספת `.take(...)` אחרי מערכי הנתונים `ds_train` ו-`ds_test`.


In [7]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size,embed_size,mask_zero=True),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

7500/7500 [==============================] - 371s 49ms/step - loss: 0.5401 - acc: 0.8079 - val_loss: 0.3780 - val_acc: 0.8822


עכשיו, כשאנחנו משתמשים במסכות, אנחנו יכולים לאמן את המודל על כל מערך הנתונים של הכותרות והתיאורים.

> **הערה**: האם שמתם לב שהשתמשנו בווקטורייזר שאומן על כותרות החדשות, ולא על כל גוף המאמר? ייתכן שזה גורם להתעלמות מחלק מהטוקנים, ולכן עדיף לאמן מחדש את הווקטורייזר. עם זאת, ההשפעה עשויה להיות קטנה מאוד, ולכן נמשיך להשתמש בווקטורייזר המאומן מראש לטובת הפשטות.


## זיכרון ארוך-טווח (LSTM)

אחת הבעיות המרכזיות של רשתות עצביות חוזרות (RNNs) היא **דעיכת גרדיאנטים**. רשתות חוזרות יכולות להיות די ארוכות, ולעיתים מתקשות להעביר את הגרדיאנטים חזרה לשכבה הראשונה של הרשת במהלך תהליך ה-backpropagation. כאשר זה קורה, הרשת אינה יכולה ללמוד קשרים בין טוקנים רחוקים. דרך אחת להימנע מבעיה זו היא להכניס **ניהול מצב מפורש** באמצעות **שערים**. שתי הארכיטקטורות הנפוצות ביותר שמכניסות שערים הן **זיכרון ארוך-טווח** (LSTM) ויחידת ממסר משולבת (GRU). כאן נתמקד ב-LSTM.

![תמונה המציגה דוגמה לתא זיכרון ארוך-טווח](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

רשת LSTM מאורגנת באופן דומה ל-RNN, אך ישנם שני מצבים שעוברים משכבה לשכבה: המצב בפועל $c$ והווקטור הנסתר $h$. בכל יחידה, הווקטור הנסתר $h_{t-1}$ משולב עם הקלט $x_t$, וביחד הם שולטים במה שקורה למצב $c_t$ ולפלט $h_{t}$ באמצעות **שערים**. לכל שער יש פונקציית הפעלה מסוג סיגמואיד (פלט בטווח $[0,1]$), שניתן לחשוב עליה כמסכה ביטית כאשר היא מוכפלת בווקטור המצב. ל-LSTM יש את השערים הבאים (משמאל לימין בתמונה למעלה):
* **שער השכחה** שקובע אילו רכיבים בווקטור $c_{t-1}$ עלינו לשכוח ואילו להעביר הלאה.
* **שער הקלט** שקובע כמה מידע מהווקטור הקלט ומהווקטור הנסתר הקודם יש לשלב בווקטור המצב.
* **שער הפלט** שלוקח את ווקטור המצב החדש ומחליט אילו מרכיביו ישמשו ליצירת הווקטור הנסתר החדש $h_t$.

ניתן לחשוב על רכיבי המצב $c$ כדגלים שניתן להפעיל ולכבות. לדוגמה, כאשר אנו נתקלים בשם *אליס* ברצף, אנו מניחים שמדובר באישה, ומפעילים את הדגל במצב שמציין שיש לנו שם עצם נשי במשפט. כאשר אנו נתקלים בהמשך במילים *וגם טום*, נפעיל את הדגל שמציין שיש לנו שם עצם ברבים. כך, באמצעות מניפולציה של המצב, ניתן לעקוב אחר התכונות הדקדוקיות של המשפט.

> **Note**: הנה משאב מצוין להבנת המבנה הפנימי של LSTM: [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) מאת כריסטופר אולה.

למרות שהמבנה הפנימי של תא LSTM עשוי להיראות מורכב, Keras מסתירה את היישום הזה בתוך שכבת `LSTM`, כך שהדבר היחיד שעלינו לעשות בדוגמה למעלה הוא להחליף את השכבה החוזרת:


In [8]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.LSTM(8),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(8),validation_data=ds_test.map(tupelize).batch(8))

15000/15000 [==============================] - 188s 13ms/step - loss: 0.5692 - acc: 0.7916 - val_loss: 0.3441 - val_acc: 0.8870


> **שימו לב** שאימון LSTMs הוא גם די איטי, ואתם עשויים לא לראות עלייה רבה בדיוק בתחילת האימון. ייתכן שתצטרכו להמשיך באימון במשך זמן מה כדי להשיג דיוק טוב.


## RNN דו-כיווני ורב-שכבתי

בדוגמאות שלנו עד כה, הרשתות החוזרות פועלות מתחילת הרצף ועד סופו. זה מרגיש טבעי לנו כי זה תואם את הכיוון שבו אנו קוראים או מקשיבים לדיבור. עם זאת, בתרחישים שבהם נדרש גישה אקראית לרצף הקלט, יש יותר היגיון להריץ את החישוב החוזר בשני הכיוונים. RNNs שמאפשרים חישובים בשני הכיוונים נקראים **RNN דו-כיווני**, וניתן ליצור אותם על ידי עטיפת השכבה החוזרת עם שכבת `Bidirectional` מיוחדת.

> **Note**: שכבת `Bidirectional` יוצרת שני עותקים של השכבה שבתוכה, ומגדירה את המאפיין `go_backwards` של אחד מהעותקים ל-`True`, כך שהוא פועל בכיוון ההפוך לאורך הרצף.

רשתות חוזרות, בין אם חד-כיווניות או דו-כיווניות, לוכדות תבניות בתוך רצף, ושומרות אותן בווקטורי מצב או מחזירות אותן כפלט. כמו ברשתות קונבולוציה, ניתן לבנות שכבה חוזרת נוספת אחרי הראשונה כדי ללכוד תבניות ברמה גבוהה יותר, שנבנות מתבניות ברמה נמוכה יותר שהשכבה הראשונה חילצה. זה מוביל אותנו למושג של **RNN רב-שכבתי**, שמורכב משתי רשתות חוזרות או יותר, כאשר הפלט של השכבה הקודמת מועבר לשכבה הבאה כקלט.

![תמונה המציגה RNN רב-שכבתי מסוג LSTM](../../../../../translated_images/he/multi-layer-lstm.dd975e29bb2a59fe.webp)

*תמונה מתוך [הפוסט הנהדר הזה](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) מאת Fernando López.*

Keras הופכת את בניית הרשתות הללו למשימה פשוטה, כי כל מה שצריך לעשות זה להוסיף עוד שכבות חוזרות למודל. עבור כל השכבות למעט האחרונה, יש להגדיר את הפרמטר `return_sequences=True`, כי אנחנו צריכים שהשכבה תחזיר את כל המצבים הביניים, ולא רק את המצב הסופי של החישוב החוזר.

בואו נבנה LSTM דו-שכבתי דו-כיווני עבור בעיית הסיווג שלנו.

> **Note** הקוד הזה שוב לוקח זמן רב להשלמה, אבל הוא נותן לנו את הדיוק הגבוה ביותר שראינו עד כה. אז אולי שווה להמתין ולראות את התוצאה.


In [9]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, 128, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64,return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),    
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

5045/7500 [===================>..........] - ETA: 2:33 - loss: 0.3709 - acc: 0.8706

## RNNs למשימות אחרות

עד כה, התמקדנו בשימוש ב-RNNs לסיווג רצפי טקסט. אך הם יכולים להתמודד עם משימות רבות נוספות, כמו יצירת טקסט ותרגום מכונה — נבחן את המשימות הללו ביחידה הבאה.



---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). בעוד שאנו שואפים לדיוק, יש להיות מודעים לכך שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור סמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
